In [1]:
import numpy as np
import ccdm
import numpy as np
from functools import reduce

In [7]:
def bound_interval_binary(seq, P_X, k):
    # seq: sequência como lista de '0' e '1'
    # P_X: dicionário com probabilidades
    # k: número de bits de entrada

    Pn = np.prod([P_X[s] for s in seq])
    ratio = 2**(-k) / Pn
    upper_bound = 1 / (P_X['0'] * P_X['1'])

    return {
        "lower_bound": 1,
        "ratio": ratio,
        "upper_bound": upper_bound
    }

def bound_interval_nonbinary(seq, P_X, k):
    Pn = np.prod([P_X[s] for s in seq])
    ratio = 2**(-k) / Pn

    p_min = min(P_X.values())
    p_max = max(P_X.values())
    upper_bound = 1 / (p_min * p_max)

    return {
        "lower_bound": 1,
        "ratio": ratio,
        "upper_bound": upper_bound
    }


def kl_upper_bound_binary(P_X):
    return np.log2(1 / (P_X['0'] * P_X['1']))

def kl_upper_bound_nonbinary(P_X):
    p_min = min(P_X.values())
    p_max = max(P_X.values())
    return -np.log2(p_min * p_max)

def prob_sequence(seq, P_X):
    return np.prod([P_X[s] for s in seq])


In [8]:
P_X = {'0': 0.4, '1': 0.6}
seq = ['1', '0', '1', '1']
k = 3  # bits de entrada

print(bound_interval_binary(seq, P_X, k))
print(kl_upper_bound_binary(P_X))
print("P(a^n) =", prob_sequence(seq, P_X))  # Output: 0.147

{'lower_bound': 1, 'ratio': 1.4467592592592595, 'upper_bound': 4.166666666666667}
2.0588936890535687
P(a^n) = 0.08639999999999999


In [9]:
def cumulative_prob(P):
    """Cria os limites cumulativos para cada símbolo"""
    cum = {}
    total = 0.0
    for symbol, prob in sorted(P.items()):
        cum[symbol] = (total, total + prob)
        total += prob
    return cum

def matching_canditade_intervals(symbol, source_interval, cum):
    """Encontra os intervalos de símbolos que coincidem com os da fonte"""
    matching = {}
    symbol_low_source, symbol_high_source = source_interval
    print(f"Intervalo Fonte: {symbol_low_source}, {symbol_high_source}")
    for cc in cum:
        symbol_low_canditate, symbol_high_canditate = cum[cc]
        print(f"Intervalo Candidato: {symbol_low_canditate}, {symbol_high_canditate}")
        if symbol_low_source >= symbol_low_canditate and symbol_high_source <= symbol_high_canditate:
            matching = {f"{symbol}": (symbol_low_canditate, symbol_high_canditate)}
            print(f"Correspondência encontrada: {matching}")
            return symbol, symbol_low_canditate, symbol_high_canditate
    return None, None, None

def rescale_source_interval(P_source, source_interval, new_low, new_high):
    """Redimensiona os intervalos cumulativos para corresponder aos da fonte"""
    
    low, high = source_interval
    range_ = new_high - new_low
    new_bounder_low = (low - new_low)/ range_
    new_bounder_high = (high - new_low) / range_
    new_bounder_low = (low - new_low)/ range_
    mid = (new_bounder_low + new_bounder_high) / 2
    P_source = {'0': (new_bounder_low, mid), '1': (mid, new_bounder_high)}
    return P_source

def update_code_interval(code_intervals, P):
    print("Atualizando intervalo de código...")
    _, m1 = code_intervals['0']
    new_code_intervals = []
    for symbol in code_intervals:
        lowerBound, upperBound  = code_intervals[symbol]
        probability = P[symbol]
        if symbol == '0':
            upperBound = m1 - (1-probability) * (m1 - lowerBound)
            print(f"Novo limite superior para {symbol}: {upperBound}")
            new_code_intervals.append((lowerBound, upperBound))
            new_code_intervals.append((upperBound, m1))
        else:
            lowerBound = m1 + (1-probability) * (upperBound - m1)
            new_code_intervals.append((m1, lowerBound))
            new_code_intervals.append((lowerBound, upperBound))
            print(f"Novo limite lowerBound para {symbol}: {lowerBound}")
    print("Novo intervalo de código:", new_code_intervals)
    return new_code_intervals

def arithmetic_encode(sequence, P):
    """Codifica uma sequência usando codificação aritmética"""
    cum = cumulative_prob(P)
    P_source = {'0': (0.0, 0.5), '1': (0.5, 1.0)}
    low = 0.0
    high = 1.0
    out = []
    print("Intervalo Candidato:", cum)
    print("Intervalo Fonte:", P_source)

    for symbol in sequence:
        source_interval = P_source[symbol]
        new_sym, new_low, new_high = matching_canditade_intervals(symbol, source_interval, cum)
        if new_sym:
            P_source = rescale_source_interval(P_source, source_interval, new_low, new_high)
            print(P_source)
            out.extend([new_sym])
        
        else: 
            x = update_code_interval(cum, P)
            
    print("CC:", out)

    # (low, high) = P_source[new_sym]
    # range_ = high - low
    # high = low + range_ * symbol_high
    # low = low + range_ * symbol_low
    # print(f"Símbolo: {symbol}, Intervalo: [{low:.10f}, {high:.10f})")

    # O valor final pode ser qualquer número entre [low, high)
    # Usamos o ponto médio como representação
    code = (low + high) / 2
    return code, (low, high)



In [10]:
# Exemplo
P = {'0': 0.4, '1': 0.6}
sequence = ['1', '0']
P_source = {'0': (0.0, 0.5), '1': (0.5, 1.0)}

matching = arithmetic_encode(sequence, P)


Intervalo Candidato: {'0': (0.0, 0.4), '1': (0.4, 1.0)}
Intervalo Fonte: {'0': (0.0, 0.5), '1': (0.5, 1.0)}
Intervalo Fonte: 0.5, 1.0
Intervalo Candidato: 0.0, 0.4
Intervalo Candidato: 0.4, 1.0
Correspondência encontrada: {'1': (0.4, 1.0)}
{'0': (0.16666666666666663, 0.5833333333333333), '1': (0.5833333333333333, 1.0)}
Intervalo Fonte: 0.16666666666666663, 0.5833333333333333
Intervalo Candidato: 0.0, 0.4
Intervalo Candidato: 0.4, 1.0
Atualizando intervalo de código...
Novo limite superior para 0: 0.16000000000000003
Novo limite lowerBound para 1: 0.64
Novo intervalo de código: [(0.0, 0.16000000000000003), (0.16000000000000003, 0.4), (0.4, 0.64), (0.64, 1.0)]
CC: ['1']
